In [1]:
from __future__ import division
import imp
import uproot
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import os
import ROOT
import time
import math
from array import array
import collections

inputdir = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/'
outputdir = inputdir+'xsec_genie_evrate_mar18/' 
output_filedir = outputdir
lib_function_dir = '/home/tmettler/Desktop/uBoone/do_plots/'

# helper functions
globale = imp.load_source('globale',lib_function_dir+'globale.py')
NuCC = imp.load_source('NuCC_function',lib_function_dir+'NuCC_function.py')
NuCC_w = imp.load_source('NuCCWeight_function',lib_function_dir+'NuCCWeight_function.py')

Welcome to JupyROOT 6.10/08


In [2]:
#!jupyter nbconvert --to script numuCCana_xsec_genie_other_momentum.ipynb

In [3]:
# initialte ROOT default canvas
ROOT.gROOT.SetBatch(ROOT.kTRUE)
ROOT.gStyle.SetOptStat(0)
c1 = ROOT.TCanvas("c1","c1",1600,1200)
c1.SetGrid(1)
c1.SetLeftMargin(0.14)
c1.SetRightMargin(0.18)
c1.SetBottomMargin(0.14)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


# Genie other 2 univers sims, systematic, Momentum

In [4]:
# Load input files
outputdir_png, outputdir_root,outputdir_pdf = NuCC.prepareOutput2(outputdir)

try:
    os.stat(output_filedir)
except:
    os.mkdir(output_filedir)
RootFile = ROOT.TFile(output_filedir+"Genie_other_evrates.root","RECREATE");

#filename_overlay = 'NuCCana_overlay_V26_weight.root'
filename_overlay = 'NuCCana_overlay_V26_mar18.root'

#filename_overlay = 'NuCCana_overlay_v25_weighted_all.root'
filename_data = 'NuCCana_data_V25.root'
filename_ext = 'NuCCana_ext_V25.root'
filename_dirt = 'NuCCana_dirt_V26_weight.root'
    
tree_name = 'numuCCAna'

In [5]:
#Open all the trees of the four files (data, ext, dirt, overlay)

data, ext, dirt, overlay = NuCC.openTrees(inputdir, filename_data, filename_ext, filename_dirt, filename_overlay, tree_name)
NuCC.printNumberOfEntries(data,ext,dirt,overlay)

pot_overlay = NuCC.getPOT(inputdir,filename_overlay,tree_name)
pot_dirt =  NuCC.getPOT(inputdir,filename_dirt,tree_name)
#V25 files
pot_data =    7.644e+18  # best with tor875
data_trigger = 1838700.0 #2220362.0 #1854495.0 #4743794 # 1987072.0 # E1DCNT_wcut
ext_trigger =  18997529.0  #2120135 #5685315 # EXT

print 'POT: '
print 'Data:\t\t', pot_data
print 'Ext:\t\t', 0
print 'Overlay:\t', pot_overlay
print 'Dirt:\t\t', pot_dirt
print ''
sample = [data,ext,overlay,dirt]
scale = {data:1.0,ext:1.0,overlay:1.0,dirt:1.0}
name = {data:'data',ext:'ext',overlay:'overlay',dirt:'dirt'}

scale[data], scale[ext], scale[dirt], scale[overlay] = NuCC.calculateScale(data_trigger, ext_trigger, pot_data, pot_dirt, pot_overlay)

scale[dirt] = scale[dirt]
scale[overlay] = scale[overlay]
print 'Scalefactors: '
print 'Data:\t\t', scale[data]
print 'Ext:\t\t', scale[ext]
print 'Overlay:\t', scale[overlay]
print 'Dirt:\t\t', scale[dirt]


Overlay: Number of Entries:	1012051
Data: Number of Entries:	28005
Ext: Number of Entries:		222702
Dirt: Number of Entries:	40808

POT: 
Data:		7.644e+18
Ext:		0
Overlay:	1.26811760704e+21
Dirt:		1.25019694173e+20

Scalefactors: 
Data:		1.0
Ext:		0.0967862715198
Overlay:	0.0060278320856
Dirt:		0.0611423668132


Warning in <TClass::Init>: no dictionary for class ROOT::TIOFeatures is available


In [6]:
if 1:
    #filename_overlay = filename_overlay+'out4.root'
    filename_overlay = 'NuCCana_overlay_V26_mar18_noflux.rootout4.root'
    #filename_overlay = 'NuCCana_overlay_points_1kev.rootout2.root'
    filename_data = filename_data+'out4.root'
    filename_ext = filename_ext+'out4.root'
    filename_dirt = filename_dirt+'out4.root'

    tree_name = 't_out'

    data_out, ext_out, dirt_out, overlay_out = NuCC.openTreesOut(inputdir, filename_data, filename_ext, filename_dirt, filename_overlay, tree_name)
    NuCC.printNumberOfEntries(data_out,ext_out,dirt_out,overlay_out)

    sample_out = [data_out,ext_out,overlay_out,dirt_out]
    scale_out = {data_out:1.0,ext_out:1.0,overlay_out:1.0,dirt_out:1.0}
    name_out = {data_out:'data',ext_out:'ext',overlay_out:'overlay',dirt_out:'dirt'}

    scale_out[data_out], scale_out[ext_out], scale_out[dirt_out], scale_out[overlay_out] = NuCC.calculateScale(data_trigger, ext_trigger, pot_data, pot_dirt, pot_overlay)
    scale_out[dirt_out] = scale_out[dirt_out]
    scale_out[overlay_out] = scale_out[overlay_out]

Overlay: Number of Entries:	1013969
Data: Number of Entries:	28004
Ext: Number of Entries:		222701
Dirt: Number of Entries:	40800



In [7]:
##### flux and number of tragets parameters###
flux = 1.16859e11/1.592e20 # flux per POT per cm2
print flux
flux = 7.3789785277e-10
print flux
roh_data = 1.3836 #data denisity g/cm3
roh_MC = 1.3954 # MC denisity g/cm3
mol = 39.95 # g for argon
N_A = 6.022140857e23 # molec/mol avogadro zahl
N_nucleons = 40.0
V_fid = ((254.8-10)-(-1.55+10))*((117.47-10)-(-115.53+10))*((1036.9-50)-(-0.1+10))
print 'Fiducial Volume: ', V_fid
##############################################

beam_flux = flux * pot_data
print 'Beam flux = {:.5e}'.format(beam_flux),' /cm2'
N_tot = roh_data*N_A*N_nucleons*V_fid/mol
print 'Number of target nuclei= {:.5e}'.format(N_tot),' /cm3'

7.34038944724e-10
7.3789785277e-10
Fiducial Volume:  49184671.35
Beam flux = 5.64049e+09  /cm2
Number of target nuclei= 4.10331e+31  /cm3


In [8]:
# Define signals

fidVol = '(Nu_Vx_sce>(-1.55+10) && Nu_Vx_sce<(254.8-10)) && (Nu_Vy_sce>(-115.53+10) && Nu_Vy_sce<(117.47-10)) &&(Nu_Vz_sce>(-0.1+10) && Nu_Vz_sce<(1036.9-50))'
MCfidVol = '(MCNu_Vx>(-1.55+10) && MCNu_Vx<(254.8-10)) && (MCNu_Vy>(-115.53+10) && MCNu_Vy<(117.47-10)) &&(MCNu_Vz>(-0.1+10) && MCNu_Vz<(1036.9-50))'
numu_signal = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG==13 && MCTrackPurity>0.5' # numu CC signal definition
numu_true = 'MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14' # numu CC signal definition
numu_nomu = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG!=13 && MCTrackPurity>0.5' # not an MC muon
numu_lowpur = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPurity<0.5' #low purity
numu_nc = 'fidVol && MCfidVol && MCNu_CCNC==1' # nutral current
numu_ov = 'fidVol && !MCfidVol' # out of fiducial
numu_other = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG!=14' # e.g anti nu or nue
#signal = 'MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14'
for x in sample:
    x.SetAlias('muon','(muon_candidate_key==track_key)')
    
num_fidVol = {}
for x in sample:
    x.SetAlias('fidVol',fidVol)
    x.SetAlias('MCfidVol',MCfidVol)
    x.SetAlias('numu_signal',numu_signal)
    x.SetAlias('numu_true',numu_true)
    x.SetAlias('numu_nomu',numu_nomu)
    x.SetAlias('numu_lowpur',numu_lowpur)
    x.SetAlias('numu_nc',numu_nc)
    x.SetAlias('numu_ov',numu_ov)
    x.SetAlias('numu_other',numu_other)
    num_fidVol[x] = x.GetEntries('fidVol && muon')*scale[x]
    
tot_num_fidVol = num_fidVol[ext]+num_fidVol[dirt]+num_fidVol[overlay]
overlay_signals = {'numu_signal','numu_nomu','numu_lowpur','numu_nc','numu_ov','numu_other'}

In [9]:
for x in sample_out:
    x.SetAlias('muon','(track_key == key_muon)')

num_fidVol = {}
for x in sample_out:
    x.SetAlias('fidVol',fidVol)
    x.SetAlias('MCfidVol',MCfidVol)
    x.SetAlias('numu_signal',numu_signal)
    x.SetAlias('numu_true',numu_true)
    x.SetAlias('numu_nomu',numu_nomu)
    x.SetAlias('numu_lowpur',numu_lowpur)
    x.SetAlias('numu_nc',numu_nc)
    x.SetAlias('numu_ov',numu_ov)
    x.SetAlias('numu_other',numu_other)

In [10]:
# Load the global variables for access of functions
NuCC.loadGlobal(data,ext,dirt,overlay,data_out,ext_out,dirt_out,overlay_out,scale,scale_out,tot_num_fidVol,overlay_signals,sample,sample_out, name,name_out, outputdir_png, outputdir_root,outputdir_pdf)
#NuCC.printGlobal()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [11]:
# initialte ROOT default canvas
ROOT.gStyle.SetOptStat(0)
c1 = ROOT.TCanvas("c1","c1",1600,1200)
c1.SetGrid(1)
c1.SetLeftMargin(0.14)
c1.SetRightMargin(0.18)
c1.SetBottomMargin(0.14)

In [12]:
'''track_start_border_x = '(TrackStart_x_sce <(-1.55+1) || TrackStart_x_sce > (254.8-1))'
track_end_border_x = '(TrackEnd_x_sce <(-1.55+1) || TrackEnd_x_sce > (254.8-1))'
track_start_border_y = '(TrackStart_y_sce <(-115.53+1) || TrackStart_y_sce > (117.47-1))'
track_end_border_y = '(TrackEnd_y_sce <(-115.53+1) || TrackEnd_y_sce > (117.47-1))'
track_start_border_z = '(TrackStart_z_sce <(0.1+1) || TrackStart_z_sce > (1036.9-1))'
track_end_border_z = '(TrackEnd_z_sce <(0.1+1) || TrackEnd_z_sce > (1039.9-1))'
'''
track_start_border_x = '(TrackStart_x_sce <(-1.55+5) || TrackStart_x_sce > (254.8-5))'
track_end_border_x = '(TrackEnd_x_sce <(-1.55+5) || TrackEnd_x_sce > (254.8-5))'
track_start_border_y = '(TrackStart_y_sce <(-115.53+5) || TrackStart_y_sce > (117.47-5))'
track_end_border_y = '(TrackEnd_y_sce <(-115.53+5) || TrackEnd_y_sce > (117.47-5))'
track_start_border_z = '(TrackStart_z_sce <(0.1+5) || TrackStart_z_sce > (1036.9-5))'
track_end_border_z = '(TrackEnd_z_sce <(0.1+5) || TrackEnd_z_sce > (1039.9-5))'

track_end_uncontained = '(' + track_end_border_x + ' || ' + track_end_border_y + ' || ' + track_end_border_z+ ')'


data.SetAlias("track_end_uncontained",track_end_uncontained)
ext.SetAlias("track_end_uncontained",track_end_uncontained)
overlay.SetAlias("track_end_uncontained",track_end_uncontained)
dirt.SetAlias("track_end_uncontained",track_end_uncontained)
data_out.SetAlias("track_end_uncontained",track_end_uncontained)
ext_out.SetAlias("track_end_uncontained",track_end_uncontained)
overlay_out.SetAlias("track_end_uncontained",track_end_uncontained)
dirt_out.SetAlias("track_end_uncontained",track_end_uncontained)

data.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-4)<0.9 || crtt0_time==-1)")
ext.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.57+3.195-4)<0.9 || crtt0_time==-1)")
overlay.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
dirt.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
data_out.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.95)<0.9 || crtt0_time==-1)")
ext_out.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.57+3.195-3.95)<0.9 || crtt0_time==-1)")
overlay_out.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
dirt_out.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")

crt_tom_cut = 'nr_crthit_top==0 && crthit_vertex_zcut==0 && (track_end_uncontained==1 || nr_crthit_beam_tres==0) && crt_cut'

data.SetAlias("crt_tom_cut",crt_tom_cut)
ext.SetAlias("crt_tom_cut",crt_tom_cut)
overlay.SetAlias("crt_tom_cut",crt_tom_cut)
dirt.SetAlias("crt_tom_cut",crt_tom_cut)
data_out.SetAlias("crt_tom_cut",crt_tom_cut)
ext_out.SetAlias("crt_tom_cut",crt_tom_cut)
overlay_out.SetAlias("crt_tom_cut",crt_tom_cut)
dirt_out.SetAlias("crt_tom_cut",crt_tom_cut)

weight_name = 'EventWeight*TunedCentralValue_Genie'


In [13]:
#weight_list = {}
'''weight_list_all = [ 'All_Genie', 'AxFFCCQEshape_Genie', 'DecayAngMEC_Genie', 'MaNCRES_Genie', 'Theta_Delta2Npi_Genie', 'TunedCentralValue_Genie',\
    'VecFFCCQEshape_Genie', 'expskin_FluxUnisim', 'horncurrent_FluxUnisim', 'kminus_PrimaryHadronNormalization', 'kplus_PrimaryHadronFeynmanScaling',\
    'kzero_PrimaryHadronSanfordWang', 'nucleoninexsec_FluxUnisim', 'nucleonqexsec_FluxUnisim', 'nucleontotxsec_FluxUnisim',\
    'piminus_PrimaryHadronSWCentralSplineVariation', 'pioninexsec_FluxUnisim', 'pionqexsec_FluxUnisim', 'piontotxsec_FluxUnisim',\
    'piplus_PrimaryHadronSWCentralSplineVariation', 'splines_general_Spline' ]'''

#weight_list_genie_other = [ 'AxFFCCQEshape_Genie', 'DecayAngMEC_Genie', 'MaNCRES_Genie', 'Theta_Delta2Npi_Genie', 'VecFFCCQEshape_Genie' ]

'''weight_list_flux = [ 'expskin_FluxUnisim', 'horncurrent_FluxUnisim', 'kminus_PrimaryHadronNormalization', 'kplus_PrimaryHadronFeynmanScaling',\
    'kzero_PrimaryHadronSanfordWang', 'nucleoninexsec_FluxUnisim', 'nucleonqexsec_FluxUnisim', 'nucleontotxsec_FluxUnisim',\
    'piminus_PrimaryHadronSWCentralSplineVariation', 'pioninexsec_FluxUnisim', 'pionqexsec_FluxUnisim', 'piontotxsec_FluxUnisim',\
    'piplus_PrimaryHadronSWCentralSplineVariation' ]'''

#weight_list = [ 'AxFFCCQEshape_Genie', 'DecayAngMEC_Genie', 'MaNCRES_Genie', 'Theta_Delta2Npi_Genie', 'VecFFCCQEshape_Genie' ]
weight_list = [ 'AxFFCCQEshape_UBGenie', 'DecayAngMEC_UBGenie', 'NormCCCOH_UBGenie', 'NormNCCOH_UBGenie',\
               'RPA_CCQE_Reduced_UBGenie','RPA_CCQE_UBGenie','ThetaDelta2NRad_UBGenie','Theta_Delta2Npi_UBGenie',\
                'VecFFCCQEshape_UBGenie']#,'XSecShape_CCMEC_UBGenie']

num_para = len(weight_list)
print 'Nuber of parameters: ',num_para


Nuber of parameters:  9


In [14]:
weight_name = 'EventWeight*TunedCentralValue_Genie'
num_universes = 2
eff_uni = np.zeros((num_para,num_universes))
mom_bins = [ 0.00, 0.18, 0.30, 0.45, 0.77, 1.28, 2.50 ]
binnum = len(mom_bins) - 1

In [15]:
variable = 'TrackMomMCS_mom'
cut = 'fidVol && muon && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'

h_ext_mom = ROOT.TH1F('h_ext_mom','h_ext_mom',binnum,array('f',mom_bins))
h_data_mom = ROOT.TH1F('h_data_mom','h_data_mom',binnum,array('f',mom_bins))
h_dirt_mom = ROOT.TH1F('h_dirt_mom','h_dirt_mom',binnum,array('f',mom_bins))

globale.ext_out.Draw(variable+'>>h_ext_mom','('+cut+')')
globale.data_out.Draw(variable+'>>h_data_mom','('+cut+')')
globale.dirt_out.Draw(variable+'>>h_dirt_mom',weight_name+'*('+cut+')')

h_data_mom.Scale(globale.scale[globale.data])
h_ext_mom.Scale(globale.scale[globale.ext])
h_dirt_mom.Scale(globale.scale[globale.dirt])
#h_overlay.Scale(globale.scale[globale.overlay])

bkg_ext = h_ext_mom.Integral(0,binnum+1)
num_data = h_data_mom.Integral(0,binnum+1)
bkg_dirt = h_dirt_mom.Integral(0,binnum+1)

print 'Data: ', num_data,' Ext: ', bkg_ext,' Dirt: ',bkg_dirt

h_data_mom.Write("h_data_mom")
h_ext_mom.Write("h_ext_mom")
h_dirt_mom.Write("h_dirt_mom")

Data:  1348.0  Ext:  142.179031156  Dirt:  32.1298251934


442

In [16]:
def make_mc_hist(cut,variable,xtitle,xstart,xend,xbins,name,weight_name):
    #weight_name = 'EventWeight*TunedCentralValue_Genie'
    h_bgk = ROOT.TH1F("h_bgk",'h_bgk',xbins,xstart,xend)
    overlay_out.Draw(variable+'>>h_bgk',weight_name+'*('+cut+')','')
    h_bgk.SetXTitle(xtitle)
    h_bgk.SetYTitle("Entries per bin")
    h_bgk.Scale(scale_out[overlay_out])
    h_bgk.Draw("")
    h_bgk.Write(name)
    return
def make_data_hist(cut,variable,xtitle,xstart,xend,xbins,name):
    h_bgk = ROOT.TH1F("h_bgk",'h_bgk',xbins,xstart,xend)
    data_out.Draw(variable+'>>h_bgk','1*('+cut+')','')
    h_bgk.SetXTitle(xtitle)
    h_bgk.SetYTitle("Entries per bin")
    h_bgk.Scale(scale_out[data_out])
    h_bgk.Draw("")
    h_bgk.Write(name+'dat')
    ext_out.Draw(variable+'>>h_bgk','1*('+cut+')','')
    h_bgk.SetXTitle(xtitle)
    h_bgk.SetYTitle("Entries per bin")
    h_bgk.Scale(scale_out[ext_out])
    h_bgk.Draw("")
    h_bgk.Write(name+'ext')
    dirt_out.Draw(variable+'>>h_bgk','EventWeight*TunedCentralValue_Genie*('+cut+')','')
    h_bgk.SetXTitle(xtitle)
    h_bgk.SetYTitle("Entries per bin")
    h_bgk.Scale(scale_out[dirt_out])
    h_bgk.Draw("")
    h_bgk.Write(name+'dirt')


In [17]:
cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'
name = '_all'

make_data_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+name)
make_data_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+name)
make_data_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name)
make_data_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name)
make_data_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name)
make_data_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+name)
make_data_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+name)
make_data_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+name)
make_data_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+name)
make_data_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+name)
make_data_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+name)
make_data_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name)
make_data_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+name)
make_data_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+name)
make_data_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+name)
make_data_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+name)

for para in range(num_para): 
    print 'Get efficiency of Parameter: ', weight_list[para]
    for uni in range(num_universes):
        # do stuff here
        this_name = name+ '[' + str(para)+']['+str(uni)+']'
        make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+this_name,weight_list[para]+str([uni]))

Get efficiency of Parameter:  AxFFCCQEshape_UBGenie
Get efficiency of Parameter:  DecayAngMEC_UBGenie
Get efficiency of Parameter:  NormCCCOH_UBGenie
Get efficiency of Parameter:  NormNCCOH_UBGenie
Get efficiency of Parameter:  RPA_CCQE_Reduced_UBGenie
Get efficiency of Parameter:  RPA_CCQE_UBGenie
Get efficiency of Parameter:  ThetaDelta2NRad_UBGenie
Get efficiency of Parameter:  Theta_Delta2Npi_UBGenie
Get efficiency of Parameter:  VecFFCCQEshape_UBGenie


In [18]:
cut_bkg = 'fidVol && muon'
name = '_pre'

make_data_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+name)
make_data_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+name)
make_data_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name)
make_data_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name)
make_data_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name)
make_data_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+name)
make_data_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+name)
make_data_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+name)
make_data_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+name)
make_data_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+name)
make_data_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+name)
make_data_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name)
make_data_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+name)
make_data_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+name)
make_data_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+name)
make_data_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+name)

for para in range(num_para): 
    print 'Get efficiency of Parameter: ', weight_list[para]
    for uni in range(num_universes):
        # do stuff here
        this_name = name+ '[' + str(para)+']['+str(uni)+']'
        make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+this_name,weight_list[para]+str([uni]))

Get efficiency of Parameter:  AxFFCCQEshape_UBGenie
Get efficiency of Parameter:  DecayAngMEC_UBGenie
Get efficiency of Parameter:  NormCCCOH_UBGenie
Get efficiency of Parameter:  NormNCCOH_UBGenie
Get efficiency of Parameter:  RPA_CCQE_Reduced_UBGenie
Get efficiency of Parameter:  RPA_CCQE_UBGenie
Get efficiency of Parameter:  ThetaDelta2NRad_UBGenie
Get efficiency of Parameter:  Theta_Delta2Npi_UBGenie
Get efficiency of Parameter:  VecFFCCQEshape_UBGenie


In [19]:
'''cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && numu_signal'
name = 'signal_all'

for para in range(num_para): 
    print 'Get efficiency of Parameter: ', weight_list[para]
    for uni in range(num_universes):
        # do stuff here
        this_name = name+ '[' + str(para)+']['+str(uni)+']'
        make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+this_name,weight_list[para]+str([uni]))'''

"cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && numu_signal'\nname = 'signal_all'\n\nfor para in range(num_para): \n    print 'Get efficiency of Parameter: ', weight_list[para]\n    for uni in range(num_universes):\n        # do stuff here\n        this_name = name+ '[' + str(para)+']['+str(uni)+']'\n        make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+this_name,weight_list[para]+str([uni]))\n        make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+this_name,weight_list[para]+str([uni]))\n        make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+this_name,weight_list[para]+str([uni]))\n        make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+this_name,weight_list[para]+str([uni]))\n        make_mc_hist(cut_bkg,'Nu_Vz

In [20]:
'''cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && !numu_signal'
name = 'bkg_all'

for para in range(num_para): 
    print 'Get efficiency of Parameter: ', weight_list[para]
    for uni in range(num_universes):
        # do stuff here
        this_name = name+ '[' + str(para)+']['+str(uni)+']'
        make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+this_name,weight_list[para]+str([uni]))'''

"cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && !numu_signal'\nname = 'bkg_all'\n\nfor para in range(num_para): \n    print 'Get efficiency of Parameter: ', weight_list[para]\n    for uni in range(num_universes):\n        # do stuff here\n        this_name = name+ '[' + str(para)+']['+str(uni)+']'\n        make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+this_name,weight_list[para]+str([uni]))\n        make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+this_name,weight_list[para]+str([uni]))\n        make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+this_name,weight_list[para]+str([uni]))\n        make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+this_name,weight_list[para]+str([uni]))\n        make_mc_hist(cut_bkg,'Nu_Vz_s

In [21]:
# close the root file with all the histos
RootFile.Close()

In [22]:
adfas

NameError: name 'adfas' is not defined

In [ ]:
cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && !numu_signal'
name = '_bkg_all'

for para in range(num_para): 
    print 'Get efficiency of Parameter: ', weight_list[para]
    for uni in range(num_universes):
        # do stuff here
        this_name = name+ '[' + str(para)+']['+str(uni)+']'
        make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+this_name,weight_list[para]+str([uni]))
        make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+this_name,weight_list[para]+str([uni]))

In [ ]:
# close the root file with all the histos
RootFile.Close()

In [ ]:
w5wret

In [ ]:


cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && !numu_signal'
name = '_bkg_all'

make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+name)
make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+name) 
make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name) 
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+name) 
make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+name) 
make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+name) 
make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+name) 
make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name) 
make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+name) 
make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+name) 
make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+name) 
make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+name) 

name = '_bkg_all_fine'
bins_ = 60
make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,bins_,'TrackMom_MCS'+name)
make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,bins_,'TrackMom_range'+name) 
make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,bins_,'NuVx_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,bins_,'NuVy_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,bins_,'NuVz_sce'+name) 
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,bins_,'cosTheta'+name) 
make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,bins_,'TrackLength'+name) 
make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,bins_,'PID_chiproton'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,bins_,'PIDmupion_ration'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,bins_,'PIDmuprot_ration'+name) 
make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,10,10,'nr_crthit_top'+name) 
make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,10,10,'CRTVertexcut'+name) 
make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,bins_,'TrackScore'+name) 
make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,bins_,'NuScore'+name) 
make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,20,20,'NuTracks'+name) 
make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,20,20,'NumPfp'+name) 

#make_mc_hist(cut_bkg,'TrackMomMCS_mom','Topological Score',0,2.5,20,'NuScore'+name) 

cut_bkg = 'fidVol && muon && !numu_signal'
name = '_bkg_pre'

make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+name)
make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+name) 
make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name) 
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+name) 
make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+name) 
make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+name) 
make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+name) 
make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name) 
make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+name) 
make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+name) 
make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+name) 
make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+name) 


name = '_bkg_pre_fine'

make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,bins_,'TrackMom_MCS'+name)
make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,bins_,'TrackMom_range'+name) 
make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,bins_,'NuVx_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,bins_,'NuVy_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,bins_,'NuVz_sce'+name) 
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,bins_,'cosTheta'+name) 
make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,bins_,'TrackLength'+name) 
make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,bins_,'PID_chiproton'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,bins_,'PIDmupion_ration'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,bins_,'PIDmuprot_ration'+name) 
make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,10,10,'nr_crthit_top'+name) 
make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,10,10,'CRTVertexcut'+name) 
make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,bins_,'TrackScore'+name) 
make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,bins_,'NuScore'+name) 
make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,20,20,'NuTracks'+name) 
make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,20,20,'NumPfp'+name) 


cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && numu_signal'
name = '_sig_all'

make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+name)
make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+name) 
make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name) 
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+name) 
make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+name) 
make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+name) 
make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+name) 
make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name) 
make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+name) 
make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+name) 
make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+name) 
make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+name) 

#make_mc_hist(cut_bkg,'TrackMomMCS_mom','Topological Score',0,2.5,20,'NuScore'+name) 

cut_bkg = 'fidVol && muon && numu_signal'
name = '_sig_pre'

make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,2.5,20,'TrackMom_MCS'+name)
make_mc_hist(cut_bkg,'TrackMomRange_mu','Muon momentum range [GeV]',0,2.5,20,'TrackMom_range'+name) 
make_mc_hist(cut_bkg,'Nu_Vx_sce','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vy_sce','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name) 
make_mc_hist(cut_bkg,'Nu_Vz_sce','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name) 
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,20,'cosTheta'+name) 
make_mc_hist(cut_bkg,'TrackLength','Track length [cm]',0,300,60,'TrackLength'+name) 
make_mc_hist(cut_bkg,'TrackPID_chiproton','Proton chi2',0,200,20,'PID_chiproton'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chipion','TrackPID_chimuon/TrackPID_chipion',0.4,1.2,20,'PIDmupion_ration'+name) 
make_mc_hist(cut_bkg,'TrackPID_chimuon/TrackPID_chiproton','TrackPID_chimuon/TrackPID_chiproton',0,0.25,25,'PIDmuprot_ration'+name) 
make_mc_hist(cut_bkg,'nr_crthit_top','Nr CRTHits in Top',0,4,4,'nr_crthit_top'+name) 
make_mc_hist(cut_bkg,'crthit_vertex_zcut','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name) 
make_mc_hist(cut_bkg,'TrackScore','TrackScore',0.5,1,20,'TrackScore'+name) 
make_mc_hist(cut_bkg,'NuScore','Muon momentum [GeV]',0,1,20,'NuScore'+name) 
make_mc_hist(cut_bkg,'NuTracks','Number of tracks',0,10,10,'NuTracks'+name) 
make_mc_hist(cut_bkg,'NumPfp','Number of PFparticles',0,10,10,'NumPfp'+name) 

#make_mc_hist(cut_bkg,'TrackMomMCS_mom','Topological Score',0,2.5,20,'NuScore'+name) 

cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && Nu_Vx_sce!=-999'
name = '_pass_all'
bins_ = 20
make_mc_hist(cut_bkg,'Nu_Vx_sce - MCNu_Vx','Reco - True: Vx',-200,200,bins_,'DiffVx'+name)
make_mc_hist(cut_bkg,'Nu_Vy_sce - MCNu_Vy','Reco - True: Vy',-200,200,bins_,'DiffVy'+name)
make_mc_hist(cut_bkg,'Nu_Vz_sce - MCNu_Vz','Reco - True: Vz',-200,200,bins_,'DiffVz'+name)
make_mc_hist(cut_bkg,'sqrt((Nu_Vx_sce-MCNu_Vx)*(Nu_Vx_sce-MCNu_Vx)+(Nu_Vy_sce-MCNu_Vy)*(Nu_Vy_sce-MCNu_Vy)+(Nu_Vz_sce-MCNu_Vz)*(Nu_Vz_sce-MCNu_Vz))','Reco - True: Vertex',0,200,bins_,'DiffVertex'+name)

cut_bkg = cut_bkg+' && MCTrackPurity>0.5'
make_mc_hist(cut_bkg,'TrackStart_x_sce - MCTrackStart_x','Reco - True: start Track x',-200,200,bins_,'DiffTrackSx'+name)
make_mc_hist(cut_bkg,'TrackStart_y_sce - MCTrackStart_y','Reco - True: start Track y',-200,200,bins_,'DiffTrackSy'+name)
make_mc_hist(cut_bkg,'TrackStart_z_sce - MCTrackStart_z','Reco - True: start Track z',-200,200,bins_,'DiffTrackSz'+name)
make_mc_hist(cut_bkg,'sqrt((TrackStart_x_sce-MCTrackStart_x)*(TrackStart_x_sce-MCTrackStart_x)+(TrackStart_y_sce-MCTrackStart_y)*(TrackStart_y_sce-MCTrackStart_y)+(TrackStart_z_sce-MCTrackStart_z)*(TrackStart_z_sce-MCTrackStart_z))','Reco - True: Track Start',0,200,bins_,'DiffTrackStart'+name)

make_mc_hist(cut_bkg,'TrackEnd_x_sce - MCTrackEnd_x','Reco - True: end Track x',-200,200,bins_,'DiffTrackEx'+name)
make_mc_hist(cut_bkg,'TrackEnd_y_sce - MCTrackEnd_y','Reco - True: end Track y',-200,200,bins_,'DiffTrackEy'+name)
make_mc_hist(cut_bkg,'TrackEnd_z_sce - MCTrackEnd_z','Reco - True: end Track z',-200,200,bins_,'DiffTrackEz'+name)
make_mc_hist(cut_bkg,'sqrt((TrackEnd_x_sce-MCTrackEnd_x)*(TrackEnd_x_sce-MCTrackEnd_x)+(TrackEnd_y_sce-MCTrackEnd_y)*(TrackEnd_y_sce-MCTrackEnd_y)+(TrackEnd_z_sce-MCTrackEnd_z)*(TrackEnd_z_sce-MCTrackEnd_z))','Reco - True: Track End',0,2000,bins_,'DiffTrackEnd'+name)

cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && Nu_Vx_sce!=-999'
name = '_pass_all_fine'
bins_ = 100
make_mc_hist(cut_bkg,'Nu_Vx_sce - MCNu_Vx','Reco - True: Vx',-50,50,bins_,'DiffVx'+name)
make_mc_hist(cut_bkg,'Nu_Vy_sce - MCNu_Vy','Reco - True: Vy',-50,50,bins_,'DiffVy'+name)
make_mc_hist(cut_bkg,'Nu_Vz_sce - MCNu_Vz','Reco - True: Vz',-50,50,bins_,'DiffVz'+name)
make_mc_hist(cut_bkg,'sqrt((Nu_Vx_sce-MCNu_Vx)*(Nu_Vx_sce-MCNu_Vx)+(Nu_Vy_sce-MCNu_Vy)*(Nu_Vy_sce-MCNu_Vy)+(Nu_Vz_sce-MCNu_Vz)*(Nu_Vz_sce-MCNu_Vz))','Reco - True: Vertex',0,50,bins_,'DiffVertex'+name)

cut_bkg = cut_bkg+' && MCTrackPurity>0.5'
make_mc_hist(cut_bkg,'TrackStart_x_sce - MCTrackStart_x','Reco - True: start Track x',-50,50,bins_,'DiffTrackSx'+name)
make_mc_hist(cut_bkg,'TrackStart_y_sce - MCTrackStart_y','Reco - True: start Track y',-50,50,bins_,'DiffTrackSy'+name)
make_mc_hist(cut_bkg,'TrackStart_z_sce - MCTrackStart_z','Reco - True: start Track z',-50,50,bins_,'DiffTrackSz'+name)
make_mc_hist(cut_bkg,'sqrt((TrackStart_x_sce-MCTrackStart_x)*(TrackStart_x_sce-MCTrackStart_x)+(TrackStart_y_sce-MCTrackStart_y)*(TrackStart_y_sce-MCTrackStart_y)+(TrackStart_z_sce-MCTrackStart_z)*(TrackStart_z_sce-MCTrackStart_z))','Reco - True: Track Start',0,50,bins_,'DiffTrackStart'+name)

make_mc_hist(cut_bkg,'TrackEnd_x_sce - MCTrackEnd_x','Reco - True: end Track x',-50,50,bins_,'DiffTrackEx'+name)
make_mc_hist(cut_bkg,'TrackEnd_y_sce - MCTrackEnd_y','Reco - True: end Track y',-50,50,bins_,'DiffTrackEy'+name)
make_mc_hist(cut_bkg,'TrackEnd_z_sce - MCTrackEnd_z','Reco - True: end Track z',-50,50,bins_,'DiffTrackEz'+name)
make_mc_hist(cut_bkg,'sqrt((TrackEnd_x_sce-MCTrackEnd_x)*(TrackEnd_x_sce-MCTrackEnd_x)+(TrackEnd_y_sce-MCTrackEnd_y)*(TrackEnd_y_sce-MCTrackEnd_y)+(TrackEnd_z_sce-MCTrackEnd_z)*(TrackEnd_z_sce-MCTrackEnd_z))','Reco - True: Track End',0,2000,bins_,'DiffTrackEnd'+name)

cut_bkg = 'fidVol && muon && Nu_Vx_sce!=-999'
name = '_pass_pre'
bins_ = 20
make_mc_hist(cut_bkg,'Nu_Vx_sce - MCNu_Vx','Reco - True: Vx',-200,200,bins_,'DiffVx'+name)
make_mc_hist(cut_bkg,'Nu_Vy_sce - MCNu_Vy','Reco - True: Vy',-200,200,bins_,'DiffVy'+name)
make_mc_hist(cut_bkg,'Nu_Vz_sce - MCNu_Vz','Reco - True: Vz',-200,200,bins_,'DiffVz'+name)
make_mc_hist(cut_bkg,'sqrt((Nu_Vx_sce-MCNu_Vx)*(Nu_Vx_sce-MCNu_Vx)+(Nu_Vy_sce-MCNu_Vy)*(Nu_Vy_sce-MCNu_Vy)+(Nu_Vz_sce-MCNu_Vz)*(Nu_Vz_sce-MCNu_Vz))','Reco - True: Vertex',0,200,bins_,'DiffVertex'+name)

cut_bkg = cut_bkg+' && MCTrackPurity>0.5'
make_mc_hist(cut_bkg,'TrackStart_x_sce - MCTrackStart_x','Reco - True: start Track x',-200,200,bins_,'DiffTrackSx'+name)
make_mc_hist(cut_bkg,'TrackStart_y_sce - MCTrackStart_y','Reco - True: start Track y',-200,200,bins_,'DiffTrackSy'+name)
make_mc_hist(cut_bkg,'TrackStart_z_sce - MCTrackStart_z','Reco - True: start Track z',-200,200,bins_,'DiffTrackSz'+name)
make_mc_hist(cut_bkg,'sqrt((TrackStart_x_sce-MCTrackStart_x)*(TrackStart_x_sce-MCTrackStart_x)+(TrackStart_y_sce-MCTrackStart_y)*(TrackStart_y_sce-MCTrackStart_y)+(TrackStart_z_sce-MCTrackStart_z)*(TrackStart_z_sce-MCTrackStart_z))','Reco - True: Track Start',0,200,bins_,'DiffTrackStart'+name)

make_mc_hist(cut_bkg,'TrackEnd_x_sce - MCTrackEnd_x','Reco - True: end Track x',-200,200,bins_,'DiffTrackEx'+name)
make_mc_hist(cut_bkg,'TrackEnd_y_sce - MCTrackEnd_y','Reco - True: end Track y',-200,200,bins_,'DiffTrackEy'+name)
make_mc_hist(cut_bkg,'TrackEnd_z_sce - MCTrackEnd_z','Reco - True: end Track z',-200,200,bins_,'DiffTrackEz'+name)
make_mc_hist(cut_bkg,'sqrt((TrackEnd_x_sce-MCTrackEnd_x)*(TrackEnd_x_sce-MCTrackEnd_x)+(TrackEnd_y_sce-MCTrackEnd_y)*(TrackEnd_y_sce-MCTrackEnd_y)+(TrackEnd_z_sce-MCTrackEnd_z)*(TrackEnd_z_sce-MCTrackEnd_z))','Reco - True: Track End',0,2000,bins_,'DiffTrackEnd'+name)

cut_bkg = 'fidVol && muon && Nu_Vx_sce!=-999'
name = '_pass_pre_fine'
bins_ = 100
make_mc_hist(cut_bkg,'Nu_Vx_sce - MCNu_Vx','Reco - True: Vx',-50,50,bins_,'DiffVx'+name)
make_mc_hist(cut_bkg,'Nu_Vy_sce - MCNu_Vy','Reco - True: Vy',-50,50,bins_,'DiffVy'+name)
make_mc_hist(cut_bkg,'Nu_Vz_sce - MCNu_Vz','Reco - True: Vz',-50,50,bins_,'DiffVz'+name)
make_mc_hist(cut_bkg,'sqrt((Nu_Vx_sce-MCNu_Vx)*(Nu_Vx_sce-MCNu_Vx)+(Nu_Vy_sce-MCNu_Vy)*(Nu_Vy_sce-MCNu_Vy)+(Nu_Vz_sce-MCNu_Vz)*(Nu_Vz_sce-MCNu_Vz))','Reco - True: Vertex',0,50,bins_,'DiffVertex'+name)

cut_bkg = cut_bkg+' && MCTrackPurity>0.5'
make_mc_hist(cut_bkg,'TrackStart_x_sce - MCTrackStart_x','Reco - True: start Track x',-50,50,bins_,'DiffTrackSx'+name)
make_mc_hist(cut_bkg,'TrackStart_y_sce - MCTrackStart_y','Reco - True: start Track y',-50,50,bins_,'DiffTrackSy'+name)
make_mc_hist(cut_bkg,'TrackStart_z_sce - MCTrackStart_z','Reco - True: start Track z',-50,50,bins_,'DiffTrackSz'+name)
make_mc_hist(cut_bkg,'sqrt((TrackStart_x_sce-MCTrackStart_x)*(TrackStart_x_sce-MCTrackStart_x)+(TrackStart_y_sce-MCTrackStart_y)*(TrackStart_y_sce-MCTrackStart_y)+(TrackStart_z_sce-MCTrackStart_z)*(TrackStart_z_sce-MCTrackStart_z))','Reco - True: Track Start',0,50,bins_,'DiffTrackStart'+name)

make_mc_hist(cut_bkg,'TrackEnd_x_sce - MCTrackEnd_x','Reco - True: end Track x',-50,50,bins_,'DiffTrackEx'+name)
make_mc_hist(cut_bkg,'TrackEnd_y_sce - MCTrackEnd_y','Reco - True: end Track y',-50,50,bins_,'DiffTrackEy'+name)
make_mc_hist(cut_bkg,'TrackEnd_z_sce - MCTrackEnd_z','Reco - True: end Track z',-50,50,bins_,'DiffTrackEz'+name)
make_mc_hist(cut_bkg,'sqrt((TrackEnd_x_sce-MCTrackEnd_x)*(TrackEnd_x_sce-MCTrackEnd_x)+(TrackEnd_y_sce-MCTrackEnd_y)*(TrackEnd_y_sce-MCTrackEnd_y)+(TrackEnd_z_sce-MCTrackEnd_z)*(TrackEnd_z_sce-MCTrackEnd_z))','Reco - True: Track End',0,2000,bins_,'DiffTrackEnd'+name)


In [ ]:
cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && !numu_signal'
name = '_bkg_all2'
make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,3,60,'TrackMom_MCS_fine'+name)
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,60,'cosTheta_fine'+name) 

cut_bkg = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && numu_signal'
name = '_sig_all'
make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,3,60,'TrackMom_MCS_fine'+name)
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,60,'cosTheta_fine'+name) 

cut_bkg = 'fidVol && muon && !numu_signal'
name = '_bkg_pre'
make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,3,60,'TrackMom_MCS_fine'+name)
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,60,'cosTheta_fine'+name) 

cut_bkg = 'fidVol && muon && numu_signal'
name = '_sig_pre'
make_mc_hist(cut_bkg,'TrackMomMCS_mom','Muon momentum MCS [GeV]',0,3,60,'TrackMom_MCS_fine'+name)
make_mc_hist(cut_bkg,'cos(TrackTheta)','cos(Theta)',-1,1,60,'cosTheta_fine'+name) 


In [ ]:
# close the root file with all the histos
RootFile.Close()